# Coding Boost: How to use Wisent to Make Your Models Better At Coding

Here, we use LiveCodeBench to compute steering vectors and change activations of a simple model to steer it in a direction of generating better code. 

In [ ]:
import os
import json

# =============================================================================
# CONFIGURATION - Base parameters (layer and strength will be optimized)
# =============================================================================

# Model Configuration
MODEL = "meta-llama/Llama-3.2-1B-Instruct"  # HuggingFace model ID

# Task Configuration
TASK = "livecodebench"             # Coding benchmark for contrastive pairs

# Steering Configuration (defaults - will be overridden by optimization)
TRAIT_LABEL = "coding_ability"     # Label for the steering vector
TOKEN_AGGREGATION = "average"      # How to aggregate token activations
NUM_PAIRS = 50                     # Number of contrastive pairs to use
NORMALIZE = True                   # Normalize steering vectors

# Generation Configuration
MAX_NEW_TOKENS = 512               # Max tokens for code generation
TEMPERATURE = 0.2                  # Lower = more deterministic
NUM_TEST_QUESTIONS = 10            # Number of problems to evaluate

# Output Configuration
OUTPUT_DIR = "./coding_boost_outputs"

# =============================================================================
# Setup - Create output directories
# =============================================================================
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/pairs", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/vectors", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/responses", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/optimization", exist_ok=True)

print("Base Configuration:")
print(f"  Model: {MODEL}")
print(f"  Task: {TASK}")
print(f"  Trait Label: {TRAIT_LABEL}")
print(f"  Num Pairs: {NUM_PAIRS}")
print(f"  Output: {OUTPUT_DIR}")
print(f"\nNote: Optimal layer and steering strength will be determined by optimization.")

## Step 1: Optimize Steering Parameters

Before creating steering vectors, we run the `optimize-steering` command to find the best layer and strength for our task. This searches across different configurations and evaluates their effectiveness.

In [ ]:
# Run steering optimization to find the best layer and strength for livecodebench
!python -m wisent.core.main optimize-steering comprehensive \
    {MODEL} \
    --tasks {TASK} \
    --methods CAA \
    --limit {NUM_PAIRS} \
    --save-best-vector {OUTPUT_DIR}/vectors \
    --verbose

In [ ]:
# Load the optimized parameters from the optimization results
import glob

# Find the most recent optimization results file
opt_results_pattern = f"./optimization_results/steering_comprehensive_*.json"
opt_results_files = glob.glob(opt_results_pattern)

if opt_results_files:
    # Load the most recent one
    latest_results = sorted(opt_results_files)[-1]
    with open(latest_results, 'r') as f:
        opt_data = json.load(f)
    
    # Get optimized parameters for our task
    task_results = opt_data.get('tasks', {}).get(TASK, {})
    
    LAYER = task_results.get('best_layer', 8)
    STEERING_STRENGTH = task_results.get('best_strength', 1.0)
    STEERING_STRATEGY = task_results.get('best_strategy', 'constant')
    TOKEN_AGGREGATION = task_results.get('best_token_aggregation', 'last_token')
    PROMPT_CONSTRUCTION = task_results.get('best_prompt_construction', 'chat_template')
    
    print("✅ Loaded optimized parameters from:", latest_results)
else:
    # Fallback defaults
    LAYER = 8
    STEERING_STRENGTH = 1.0
    STEERING_STRATEGY = 'constant'
    TOKEN_AGGREGATION = 'last_token'
    PROMPT_CONSTRUCTION = 'chat_template'
    print("⚠️ No optimization results found, using defaults")

print("\nOptimized Steering Parameters:")
print(f"  Layer: {LAYER}")
print(f"  Strength: {STEERING_STRENGTH}")
print(f"  Strategy: {STEERING_STRATEGY}")
print(f"  Token Aggregation: {TOKEN_AGGREGATION}")
print(f"  Prompt Construction: {PROMPT_CONSTRUCTION}")
print(f"\nThese parameters were determined by searching across configurations.")

## Step 2: Extract Contrastive Pairs and Generate Steering Vector

LiveCodeBench provides pre-computed code solutions with pass/fail labels. We extract:
- **Positive (correct)**: Code solutions that pass all test cases
- **Negative (incorrect)**: Code solutions that fail test cases

In [2]:
# Extract contrastive pairs from LiveCodeBench
!python -m wisent.core.main generate-pairs-from-task \
    livecodebench \
    --output {OUTPUT_DIR}/pairs/livecodebench_pairs.json \
    --limit 100 \
    --verbose

  .................  .:--++*##%%%%##**+=-:.  .................  
  ..             .:=*%@@@@@@@%%%%%%%@@@@@@%*=:.             ..  
  .           .-*%@@@%#+=-::.........:-=+#%@@@%*=.           .  
  .         -*%@@@#=:.                    .:=*%@@@*-.        .  
  .      .-#@@@*=.                            .-*@@@#-.      .  
  .     :#@@@*:                                  :+%@@#-     .  
  .   .+@@@*:                                      :+@@@+.   .  
  .  .*@@@@%*=:.                                     -%@@#:  .  
  . .#@@#=*%@@@%*-:.                                  .#@@%: .  
  ..*@@%.  .-+#@@@@#+-:.                               .*@@%..  
  .=@@@-       :-+#@@@@%*=:.                            .%@@*.  
  :#@@+           .:-+#@@@@%#+=:.                        -@@@-  
  =@@@:                .-=*%@@@@%#+=:..                  .#@@+  
  +@@@*=:.                 .:-+*%@@@@%#*=-:..             *@@+  
  +@@@@@@#+-..                  .:-=*#@@@@@%#*+--..       +@@+  
  +@@#-+%@@@%:           

In [3]:
# Examine the extracted pairs
with open(f"{OUTPUT_DIR}/pairs/livecodebench_pairs.json", 'r') as f:
    pairs_data = json.load(f)

print(f"Extracted {pairs_data['num_pairs']} contrastive pairs from {pairs_data['task_name']}")
print("\n" + "="*80)

# Show a few examples
for i, pair in enumerate(pairs_data['pairs'][:2]):
    print(f"\nExample {i+1}:")
    print(f"Problem (truncated): {pair['prompt'][:200]}...")
    print(f"\nCorrect Code (truncated):")
    print(pair['positive_response']['model_response'][:300])
    print(f"\nIncorrect Code (truncated):")
    print(pair['negative_response']['model_response'][:300])
    print("="*80)

Extracted 93 contrastive pairs from livecodebench


Example 1:
Problem (truncated): Question: There are three cards with letters $\texttt{a}$, $\texttt{b}$, $\texttt{c}$ placed in a row in some order. You can do the following operation at most once: 

 
-  Pick two cards, and swap th...

Correct Code (truncated):

t = int(input())
for _ in range(t):
    s = input().strip()
    if s == 'abc':
        print("YES")
        continue
    chars = list(s)
    found = False
    # Check all possible single swaps
    for i, j in [(0, 1), (0, 2), (1, 2)]:
        temp = chars.copy()
        temp[i], temp[j] = temp[j], 

Incorrect Code (truncated):
# Read the number of test cases
t = int(input())

for _ in range(t):
    # Read the input string
    s = input()
    
    # Check if the string is already "abc"
    if s == "abc":
        print("YES")
        continue
    
    # Check if the string can be converted to "abc" by swapping two character

Example 2:
Problem (truncated): Question: Slavic is p

### Generate Steering Vector

The `generate-vector-from-task` command runs the complete pipeline using our **optimized layer**:
1. Extract contrastive pairs
2. Collect activations from the model
3. Create steering vectors using CAA (Contrastive Activation Addition)

In [ ]:
# Generate steering vector using optimized parameters
!python -m wisent.core.main generate-vector-from-task \
    --task {TASK} \
    --trait-label {TRAIT_LABEL} \
    --model {MODEL} \
    --num-pairs {NUM_PAIRS} \
    --layers {LAYER} \
    --token-aggregation {TOKEN_AGGREGATION} \
    --output {OUTPUT_DIR}/vectors/coding_vector.pt \
    --keep-intermediate \
    --intermediate-dir {OUTPUT_DIR}/vectors \
    --normalize \
    --verbose \
    --timing

In [5]:
# Examine the generated steering vector
import torch

vector_data = torch.load(f"{OUTPUT_DIR}/vectors/coding_vector.pt")

print("Steering Vector Info:")
print(f"  Model: {vector_data.get('model', 'N/A')}")
print(f"  Trait: {vector_data.get('trait_label', 'N/A')}")
print(f"  Method: {vector_data.get('method', 'N/A')}")
print(f"  Layer: {vector_data.get('layer', 'N/A')}")

if 'steering_vector' in vector_data:
    sv = vector_data['steering_vector']
    print(f"  Vector shape: {sv.shape}")
    print(f"  Vector norm: {torch.norm(sv).item():.4f}")

Steering Vector Info:
  Model: meta-llama/Llama-3.2-1B-Instruct
  Trait: livecodebench
  Method: caa
  Layer: 8
  Vector shape: torch.Size([2048])
  Vector norm: 1.0000


## Step 3: Test Steering on a Sample Problem

Use the `multi-steer` command to apply the optimized coding steering vector during generation.

In [ ]:
# Load a test prompt dynamically from LiveCodeBench
from wisent.core.tasks import LiveCodeBenchTask

# Load LiveCodeBench data
lcb_task = LiveCodeBenchTask()
lcb_data = lcb_task.load_data(limit=10)

# Get the first problem as our test prompt
test_problem = lcb_data[0]
TEST_PROMPT = f"Question: {test_problem['question_content']}\n\nWrite a solution:"

print(f"Loaded problem: {test_problem.get('question_title', 'Unknown')}")
print(f"Prompt preview: {TEST_PROMPT[:300]}...")
print(f"\nUsing optimized parameters: layer={LAYER}, strength={STEERING_STRENGTH}")

### Baseline (no steering)

In [ ]:
# Generate baseline response (no steering)
!python -m wisent.core.main multi-steer \
    --model {MODEL} \
    --layer {LAYER} \
    --prompt "{TEST_PROMPT}" \
    --max-new-tokens {MAX_NEW_TOKENS} \
    --verbose

### Steered (using optimized parameters)

In [ ]:
# Apply steering using optimized parameters
!python -m wisent.core.main multi-steer \
    --vector {OUTPUT_DIR}/vectors/coding_vector.pt:{STEERING_STRENGTH} \
    --model {MODEL} \
    --layer {LAYER} \
    --prompt "{TEST_PROMPT}" \
    --max-new-tokens {MAX_NEW_TOKENS} \
    --verbose

In [ ]:
# Generate baseline responses (without steering) for batch evaluation
!python -m wisent.core.main generate-responses \
    {MODEL} \
    --task {TASK} \
    --output {OUTPUT_DIR}/responses/baseline_responses.json \
    --num-questions {NUM_TEST_QUESTIONS} \
    --max-new-tokens {MAX_NEW_TOKENS} \
    --temperature {TEMPERATURE} \
    --verbose

In [ ]:
# Generate steered responses using optimized parameters
# Note: generate-responses --use-steering needs vector/layer/strength params
# For now, we demonstrate with multi-steer which properly supports steering
print(f"Steered generation would use: layer={LAYER}, strength={STEERING_STRENGTH}")
print(f"Vector: {OUTPUT_DIR}/vectors/coding_vector.pt")

# TODO: Update generate-responses CLI to accept --vector, --layer, --strength params
!python -m wisent.core.main generate-responses \
    {MODEL} \
    --task {TASK} \
    --output {OUTPUT_DIR}/responses/steered_responses.json \
    --num-questions {NUM_TEST_QUESTIONS} \
    --max-new-tokens {MAX_NEW_TOKENS} \
    --temperature {TEMPERATURE} \
    --use-steering \
    --verbose

## Step 4: Evaluate Results

The `evaluate-responses` command can execute generated code in Docker to verify correctness.

In [14]:
# Evaluate baseline responses
!python -m wisent.core.main evaluate-responses \
    --input {OUTPUT_DIR}/responses/baseline_responses.json \
    --output {OUTPUT_DIR}/responses/baseline_evaluation.json \
    --task livecodebench \
    --verbose

  .................  .:--++*##%%%%##**+=-:.  .................  
  ..             .:=*%@@@@@@@%%%%%%%@@@@@@%*=:.             ..  
  .           .-*%@@@%#+=-::.........:-=+#%@@@%*=.           .  
  .         -*%@@@#=:.                    .:=*%@@@*-.        .  
  .      .-#@@@*=.                            .-*@@@#-.      .  
  .     :#@@@*:                                  :+%@@#-     .  
  .   .+@@@*:                                      :+@@@+.   .  
  .  .*@@@@%*=:.                                     -%@@#:  .  
  . .#@@#=*%@@@%*-:.                                  .#@@%: .  
  ..*@@%.  .-+#@@@@#+-:.                               .*@@%..  
  .=@@@-       :-+#@@@@%*=:.                            .%@@*.  
  :#@@+           .:-+#@@@@%#+=:.                        -@@@-  
  =@@@:                .-=*%@@@@%#+=:..                  .#@@+  
  +@@@*=:.                 .:-+*%@@@@%#*=-:..             *@@+  
  +@@@@@@#+-..                  .:-=*#@@@@@%#*+--..       +@@+  
  +@@#-+%@@@%:           

In [15]:
# Evaluate steered responses
!python -m wisent.core.main evaluate-responses \
    --input {OUTPUT_DIR}/responses/steered_responses.json \
    --output {OUTPUT_DIR}/responses/steered_evaluation.json \
    --task livecodebench \
    --verbose

  .................  .:--++*##%%%%##**+=-:.  .................  
  ..             .:=*%@@@@@@@%%%%%%%@@@@@@%*=:.             ..  
  .           .-*%@@@%#+=-::.........:-=+#%@@@%*=.           .  
  .         -*%@@@#=:.                    .:=*%@@@*-.        .  
  .      .-#@@@*=.                            .-*@@@#-.      .  
  .     :#@@@*:                                  :+%@@#-     .  
  .   .+@@@*:                                      :+@@@+.   .  
  .  .*@@@@%*=:.                                     -%@@#:  .  
  . .#@@#=*%@@@%*-:.                                  .#@@%: .  
  ..*@@%.  .-+#@@@@#+-:.                               .*@@%..  
  .=@@@-       :-+#@@@@%*=:.                            .%@@*.  
  :#@@+           .:-+#@@@@%#+=:.                        -@@@-  
  =@@@:                .-=*%@@@@%#+=:..                  .#@@+  
  +@@@*=:.                 .:-+*%@@@@%#*=-:..             *@@+  
  +@@@@@@#+-..                  .:-=*#@@@@@%#*+--..       +@@+  
  +@@#-+%@@@%:           

In [13]:
# Compare evaluation results
def load_eval_results(path):
    try:
        with open(path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return None

baseline_eval = load_eval_results(f"{OUTPUT_DIR}/responses/baseline_evaluation.json")
steered_eval = load_eval_results(f"{OUTPUT_DIR}/responses/steered_evaluation.json")

print("Evaluation Results Comparison:")
print("="*60)

if baseline_eval:
    metrics = baseline_eval.get('aggregated_metrics', {})
    print(f"\nBaseline Model:")
    print(f"  Pass Rate: {metrics.get('pass_rate', 0):.2%}")
    print(f"  Total Passed: {metrics.get('total_passed', 0)}")
    print(f"  Total Problems: {metrics.get('total_problems', 0)}")

if steered_eval:
    metrics = steered_eval.get('aggregated_metrics', {})
    print(f"\nSteered Model:")
    print(f"  Pass Rate: {metrics.get('pass_rate', 0):.2%}")
    print(f"  Total Passed: {metrics.get('total_passed', 0)}")
    print(f"  Total Problems: {metrics.get('total_problems', 0)}")

Evaluation Results Comparison:
